<a href="https://colab.research.google.com/github/phantasy77/first_study/blob/main/our_model_my.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 두 가지 카테고리 분류
# zzllrr imagergeek 다운로드
# 수집하고 싶은 이미지 검색
# 우측 상단 아이콘 클릭 -> Go 버튼 누른후 화면 전환
# 필요 없는 이미지, 아이콘 파일 클릭해서 제거
# rename 버튼 클릭후 prefix버튼 클릭 이름 명명 규칙 수정
# 수정한 이미지 이름 preview 버튼 클릭후 반영 확인
# save 누른 후 수집 대기 -> 수집 완료 후
# 해당 카테고리 폴더 생성후 모두 저장

In [2]:
# 목표 성능 90%
# 현재 89.8% -> HT
# 현재 85%
# 1. DATA
# 2. Model 교체 혹은 확장
# 3. HT

In [3]:
!unzip /content/drive/MyDrive/Colab/our_dataset.zip

Archive:  /content/drive/MyDrive/Colab/our_dataset.zip
   creating: our_dataset/
   creating: our_dataset/cute/
  inflating: our_dataset/cute/cute-001.jpeg  
  inflating: our_dataset/cute/cute-002.jpeg  
  inflating: our_dataset/cute/cute-003.jpeg  
  inflating: our_dataset/cute/cute-004.jpeg  
  inflating: our_dataset/cute/cute-005.jpeg  
  inflating: our_dataset/cute/cute-006.jpeg  
  inflating: our_dataset/cute/cute-007.jpeg  
  inflating: our_dataset/cute/cute-008.jpeg  
  inflating: our_dataset/cute/cute-009.jpeg  
  inflating: our_dataset/cute/cute-010.jpeg  
  inflating: our_dataset/cute/cute-011.jpeg  
  inflating: our_dataset/cute/cute-012.jpeg  
  inflating: our_dataset/cute/cute-013.jpeg  
  inflating: our_dataset/cute/cute-014.jpeg  
  inflating: our_dataset/cute/cute-015.jpeg  
  inflating: our_dataset/cute/cute-016.jpeg  
  inflating: our_dataset/cute/cute-017.jpeg  
  inflating: our_dataset/cute/cute-018.jpeg  
  inflating: our_dataset/cute/cute-019.jpeg  
  inflating: o

In [2]:
import os
a = '/content/'
b = 'test'
os.path.join(a,b)

'/content/test'

In [3]:
from PIL import Image
import numpy as np

# 파일을 접근 -> 메모리에 파일을 올려야 -> 모든 파일의 경로가 필요
from glob import glob
mother_path = '/content/our_dataset/'
# glob(mother_path + '/*/*')

folder_list = ['cute', 'wild']
train = []
label = []
for idx , folder_name in enumerate(folder_list):
    folder = os.path.join(mother_path, folder_name)
    all_files_path = glob(folder+'/*')
    print(all_files_path)
    for k,file_path in enumerate(all_files_path):
      img = Image.open(file_path)
      # print(img)
      img = img.resize((256,256))
      img = img.convert('RGB')
      np_img = np.array(img)
      train.append(np_img)
      label.append(idx)
      # print(train)
      # print(label)
      if k % 10 == 0:
          print(f'{k} / {len(all_files_path)}')
np_train = np.array(train)
np_label = np.array(label)

['/content/our_dataset/cute/cute_3-067.jpeg', '/content/our_dataset/cute/cute-394.jpeg', '/content/our_dataset/cute/cute-155.jpeg', '/content/our_dataset/cute/cute-367.jpeg', '/content/our_dataset/cute/cute-470.jpeg', '/content/our_dataset/cute/cute-231.jpeg', '/content/our_dataset/cute/cute-071.jpeg', '/content/our_dataset/cute/cute_4-38.jpeg', '/content/our_dataset/cute/cute_2-233.jpg', '/content/our_dataset/cute/cute_2-119.jpg', '/content/our_dataset/cute/cute-055.jpeg', '/content/our_dataset/cute/cute-180.jpeg', '/content/our_dataset/cute/cute-126.jpeg', '/content/our_dataset/cute/cute_4-84.jpeg', '/content/our_dataset/cute/cute_2-084.jpg', '/content/our_dataset/cute/cute_4-60.jpeg', '/content/our_dataset/cute/cute-05_08.jpeg', '/content/our_dataset/cute/cute-119.jpeg', '/content/our_dataset/cute/cute-058.jpeg', '/content/our_dataset/cute/cute_4-64.jpeg', '/content/our_dataset/cute/cute-319.jpeg', '/content/our_dataset/cute/cute-177.jpeg', '/content/our_dataset/cute/cute-493.jpeg',

In [4]:
print(np.bincount(np_label))
print(np_train.shape)

[1071 1042]
(2113, 256, 256, 3)


In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np_train, np_label, test_size=0.2)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(1690, 256, 256, 3) (1690,)
(423, 256, 256, 3) (423,)


In [8]:
# xy = (x_train, y_train)
# np.save('data.npy',xy)
# np.load('data.npy')

In [6]:
x_train = x_train / 255.
x_test = x_test / 255.

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images horizontally
    vertical_flip=False, # Don't randomly flip images vertically
)

# import matplotlib.pyplot as plt
# import numpy as np
batch_size = 32
img_iter = datagen.flow(x_train, y_train, batch_size=batch_size)
datagen.fit(x_train)


In [20]:
import tensorflow as tf
from tensorflow.keras import layers, models

input_layer = layers.Input(shape=(256,256,3), name='start layer')
tower_1 = layers.Conv2D(64 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu',name = 'tower_1' )(input_layer)
tower_2 = layers.Conv2D(64 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu' , name= 'tower_2_1')(input_layer)
tower_2 = layers.Conv2D(64 , kernel_size=(3,3) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_2')(tower_2)
tower_3 = layers.Conv2D(64 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_3_1')(input_layer)
tower_3 = layers.Conv2D(64 , kernel_size=(5,5) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_3_2')(tower_3)
tower_4 = layers.MaxPool2D(pool_size=(3,3) , strides=(1,1) ,
                           padding='same', name= 'tower_4_1')(input_layer)
tower_4 = layers.Conv2D(64 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_4_2')(tower_4)
#
concat_layer = layers.concatenate([tower_1,tower_2,tower_3,tower_4],axis=3)
concat_layer = layers.MaxPool2D(pool_size=(3,3) , strides=(2,2), padding='valid')(concat_layer)
##
tower_2_1 = layers.Conv2D(128 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu',name = 'tower_2_1_1' )(concat_layer)
tower_2_2_1 = layers.Conv2D(128 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu' , name= 'tower_2_2_1')(concat_layer)
tower_2_2_2 = layers.Conv2D(128 , kernel_size=(3,3) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_2_2')(tower_2_2_1)
tower_2_3_1 = layers.Conv2D(128 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_3_1')(concat_layer)
tower_2_3_2 = layers.Conv2D(128 , kernel_size=(5,5) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_3_2')(tower_2_3_1)
tower_2_4_1 = layers.MaxPool2D(pool_size=(3,3) , strides=(1,1) ,
                           padding='same', name= 'tower_2_4_1')(concat_layer)
tower_2_4_2 = layers.Conv2D(128 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_2_4_2')(tower_2_4_1)
concat_layer_2 = layers.concatenate([tower_2_1,tower_2_2_2,tower_2_3_2,tower_2_4_2],
                                  axis=3)
concat_layer_2 = layers.MaxPool2D(pool_size=(3,3) , strides=(2,2), padding='valid')(concat_layer_2)
##
tower_3_1 = layers.Conv2D(256 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu',name = 'tower_3_1_1' )(concat_layer_2)
tower_3_2_1 = layers.Conv2D(256 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu' , name= 'tower_3_2_1')(concat_layer_2)
tower_3_2_2 = layers.Conv2D(256 , kernel_size=(3,3) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_3_2_2')(tower_3_2_1)
tower_3_3_1 = layers.Conv2D(256 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_3_3_1')(concat_layer_2)
tower_3_3_2 = layers.Conv2D(256 , kernel_size=(5,5) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_3_3_2')(tower_3_3_1)
tower_3_4_1 = layers.MaxPool2D(pool_size=(3,3) , strides=(1,1) ,
                           padding='same', name= 'tower_3_4_1')(concat_layer_2)
tower_3_4_2 = layers.Conv2D(256 , kernel_size=(1,1) ,
                        padding='same', strides=(1,1),
                        activation='relu', name= 'tower_3_4_2')(tower_3_4_1)
concat_layer_3 = layers.concatenate([tower_3_1,tower_3_2_2,tower_3_3_2,tower_3_4_2],
                                  axis=3)
concat_layer_3 = layers.MaxPool2D(pool_size=(3,3) , strides=(2,2), padding='valid')(concat_layer_3)
#
last_avg_pool = layers.AveragePooling2D(pool_size=(31,31) , strides=(1,1), padding='valid')(concat_layer_3)
flat_layer = layers.Flatten()(last_avg_pool)
output_layer = layers.Dense(1 ,activation='sigmoid')(flat_layer)  # linear로 나두고 loss fn에 맡길 수도 있다
# 변경 포인트 input, last_avg_pool,output layer

model = models.Model(input_layer,output_layer)
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 start layer (InputLayer)    [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 tower_2_1 (Conv2D)          (None, 256, 256, 64)         256       ['start layer[0][0]']         
                                                                                                  
 tower_3_1 (Conv2D)          (None, 256, 256, 64)         256       ['start layer[0][0]']         
                                                                                                  
 tower_4_1 (MaxPooling2D)    (None, 256, 256, 3)          0         ['start layer[0][0]']         
                                                                                            

In [21]:
!mkdir tensorboard_log

mkdir: cannot create directory ‘tensorboard_log’: File exists


In [21]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',metrics=['accuracy'])
tb = tf.keras.callbacks.TensorBoard(log_dir='/content/tensorboard_log',
                                    histogram_freq=1)
# model.fit(x_train,y_train,
#           validation_data=(x_test,y_test),
#           epochs=10,callbacks=[tb])
model.fit(img_iter, steps_per_epoch=len(x_train)/batch_size,
          validation_data=(x_test,y_test),
          epochs=10,callbacks=[tb])
# model.save('our_model')

Epoch 1/10
52/52 [==============================] - 95s 1s/step - loss: 0.7005 - accuracy: 0.5195 - val_loss: 0.6870 - val_accuracy: 0.5437
Epoch 2/10
52/52 [==============================] - 68s 1s/step - loss: 0.6769 - accuracy: 0.5716 - val_loss: 0.6455 - val_accuracy: 0.6265
Epoch 3/10
52/52 [==============================] - 67s 1s/step - loss: 0.6722 - accuracy: 0.5870 - val_loss: 0.6613 - val_accuracy: 0.6123
Epoch 4/10
52/52 [==============================] - 67s 1s/step - loss: 0.6507 - accuracy: 0.6219 - val_loss: 0.6560 - val_accuracy: 0.5934
Epoch 5/10
52/52 [==============================] - 67s 1s/step - loss: 0.6373 - accuracy: 0.6385 - val_loss: 0.6261 - val_accuracy: 0.6312
Epoch 6/10
52/52 [==============================] - 67s 1s/step - loss: 0.6354 - accuracy: 0.6343 - val_loss: 0.6258 - val_accuracy: 0.6336
Epoch 7/10
52/52 [==============================] - 67s 1s/step - loss: 0.6458 - accuracy: 0.6254 - val_loss: 0.6602 - val_accuracy: 0.6265
Epoch 8/10
52/52 [==

In [13]:
from google.colab import files
f = files.upload()
list(f.keys())[0]

Saving wild_2-008.jpg to wild_2-008.jpg


'wild_2-008.jpg'

In [14]:
test_file_name = list(f.keys())[0]

In [22]:
img = Image.open(test_file_name)
img = img.resize((256,256))
img = img.convert('RGB')
data = np.asarray(img)
test_image_tensor = np.array([data])
test_image_tensor = test_image_tensor/255.
test_image_tensor.shape

(1, 256, 256, 3)

In [23]:
result = model.predict(test_image_tensor)
print(result[0][0])
if result[0][0] > 0.5:
    print(f'{folder_list[1]} 일 확률이 {result[0][0] * 100} % 입니다')
else:
    print(f'{folder_list[0]} 일 확률이 {(1 - result[0][0])* 100} % 입니다')


1/1 [==============================] - 1s 597ms/step
0.6252056
wild 일 확률이 62.52055764198303 % 입니다
